In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rheumatoid_Arthritis"
cohort = "GSE42842"

# Input paths
in_trait_dir = "../../input/GEO/Rheumatoid_Arthritis"
in_cohort_dir = "../../input/GEO/Rheumatoid_Arthritis/GSE42842"

# Output paths
out_data_file = "../../output/preprocess/Rheumatoid_Arthritis/GSE42842.csv"
out_gene_data_file = "../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE42842.csv"
out_clinical_data_file = "../../output/preprocess/Rheumatoid_Arthritis/clinical_data/GSE42842.csv"
json_path = "../../output/preprocess/Rheumatoid_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The predictive biomarkers of TNFα-blocking agent responsiveness are specific for adalimumab and etanercept"
!Series_summary	"The objective of this study was to identify specific gene expression profiles able to predict the response of rheumatoid arthritis patients treated with methotrexate (MTX)/adalimumab (ADA) or MTX/etanercept (ETA). Twenty RA patients were received subcutaneously Adalimumab (40 mg each other week) and eleven RA patients were received Etanercept (50 mg per week). The drug efficacy was evaluated with the DAS28 score after 3 months of treatment according to the EULAR response criteria. A blood sample was carried out in patients just before the first injection of treatment in order to isolate peripheral blood mononuclear cells (PBMC) and extract total RNA."
!Series_overall_design	"Two color experiments : patient(Cy5)/Control pool (Cy3)."
Sample Characteristics Dictionary:
{0: ['gender: M', 'gender: F'], 1: ['cell type: PBMC'], 2: 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information mentioning gene expression profiling in PBMCs, 
# this dataset contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# The trait of interest appears to be treatment response/efficacy (key 4)
trait_row = 4
age_row = None  # Age information is not provided in the sample characteristics
gender_row = 0  # Gender is available in key 0

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert response efficacy to binary: 1 for good response, 0 for moderate response"""
    if pd.isna(value):
        return None
    value = str(value).lower()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "response" in value and "moderate" not in value:
        return 1  # Good response
    elif "moderate response" in value:
        return 0  # Moderate response
    else:
        return None

def convert_age(value):
    """Convert age to continuous, but age data is not available in this dataset"""
    return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male"""
    if pd.isna(value):
        return None
    value = str(value).lower()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value == "f" or value == "female":
        return 0
    elif value == "m" or value == "male":
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Note: We can't execute step 4 (Clinical Feature Extraction) 
# because the clinical data from the previous step is not available.
# This will be handled in a future step once we have the actual data.


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# We've already seen the gene identifiers from a previous step
# The output showed numeric identifiers: '1', '2', '3', etc.
# These are likely probe IDs rather than human gene symbols
# Human gene symbols would be alphabetic identifiers like BRCA1, TP53, etc.

# Let's try to look for additional files in the cohort directory to confirm
import os

# List files in the cohort directory
files_in_dir = os.listdir(in_cohort_dir)
print(f"Files in directory: {files_in_dir}")

# Based on the previous output showing numeric identifiers ('1', '2', '3', etc.)
# and biomedical knowledge that human gene symbols are alphabetic (like BRCA1, TNF, etc.),
# it's clear these are probe IDs that will need mapping to gene symbols

requires_gene_mapping = True


Files in directory: ['GSE42842_family.soft.gz', 'GSE42842_series_matrix.txt.gz']


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['266', '266', '266', '266', '266'], 'ROW': [170.0, 168.0, 166.0, 164.0, 162.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan], 'SPOT_ID.1': [nan, nan, nan, nan, nan], 'ORDER': [1.0, 2.0, 3.

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the appropriate columns for gene ID mapping
# From the previews we can see:
# - The gene expression data index contains numeric IDs ('1', '2', '3', etc.)
# - The gene annotation dataframe has an 'ID' column with the same identifiers
# - The gene symbols should be in the 'GENE_SYMBOL' column

# 2. Get a gene mapping dataframe from the gene annotation dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# Let's examine what we have in the mapping
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Verify the result
print("\nGene expression after mapping:")
print(f"Number of genes: {len(gene_data.index)}")
print("First few gene symbols:")
print(gene_data.index[:10])


Gene mapping preview:
{'ID': ['12', '14', '15', '16', '18'], 'Gene': ['APOBEC3B', 'ATP11B', 'LOC100132006', 'DNAJA1', 'EHMT2']}

Gene expression after mapping:
Number of genes: 18379
First few gene symbols:
Index(['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT',
       'A4GNT', 'AAAS'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. First, we need to extract clinical features since we missed this step earlier
selected_clinical_data = geo_select_clinical_features(
    clinical_data, 
    trait, 
    trait_row, 
    convert_trait,
    age_row, 
    convert_age,
    gender_row, 
    convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_data))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_data.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 2. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
# Note: Already normalized in step 7
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 3. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
linked_data = geo_link_clinical_genetic_data(selected_clinical_data, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview:")
print(preview_df(linked_data))

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Gene mapping was limited to a few recognized genes (TP53, BRCA1, BRCA2, IL6, IL1B, TNF)"
)

# 7. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Usable linked data saved to {out_data_file}")
else:
    print("Linked data was not usable and was not saved.")

Clinical data preview:
{'GSM1051243': [0.0, 1.0], 'GSM1051244': [0.0, 0.0], 'GSM1051245': [0.0, 0.0], 'GSM1051246': [0.0, 0.0], 'GSM1051247': [0.0, 0.0], 'GSM1051248': [0.0, 1.0], 'GSM1051249': [0.0, 0.0], 'GSM1051250': [0.0, 0.0], 'GSM1051251': [0.0, 0.0], 'GSM1051252': [1.0, 1.0], 'GSM1051253': [1.0, 0.0], 'GSM1051254': [1.0, 1.0], 'GSM1051255': [1.0, 0.0], 'GSM1051256': [1.0, 0.0], 'GSM1051257': [1.0, 0.0], 'GSM1051258': [1.0, 0.0], 'GSM1051259': [1.0, 0.0], 'GSM1051260': [1.0, 0.0], 'GSM1051261': [1.0, 0.0], 'GSM1051262': [1.0, 1.0], 'GSM1051263': [0.0, 1.0], 'GSM1051264': [0.0, 1.0], 'GSM1051265': [0.0, 1.0], 'GSM1051266': [0.0, 0.0], 'GSM1051267': [0.0, 0.0], 'GSM1051268': [0.0, 0.0], 'GSM1051269': [0.0, 1.0], 'GSM1051270': [0.0, 0.0], 'GSM1051271': [1.0, 0.0], 'GSM1051272': [1.0, 0.0], 'GSM1051273': [1.0, 1.0]}
Clinical data saved to ../../output/preprocess/Rheumatoid_Arthritis/clinical_data/GSE42842.csv


Normalized gene data saved to ../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE42842.csv
Linked data shape: (31, 18381)
Linked data preview:


{'Rheumatoid_Arthritis': [0.0, 0.0, 0.0, 0.0, 0.0], 'Gender': [1.0, 0.0, 0.0, 0.0, 0.0], 'A1BG': [0.774057349, 0.9458077, 0.333951153, -0.318837014, 0.721209078], 'A1CF': [-1.89074717, -3.060116999, 0.48422181, 0.109527232, -0.179079622], 'A2BP1': [-0.09551822299999999, -1.495824104, 1.80468077, 1.760028199, 3.359378658], 'A2LD1': [0.102329861, 0.189160714, -0.530109012, 0.537096695, 0.461395279], 'A2M': [0.529700163, 0.276743215, -1.714299727, 0.876154117, -0.593671956], 'A2ML1': [0.848918043, 0.640742658, 0.177635878, 1.130402836, 1.254657503], 'A3GALT2': [0.128658928, 0.212888082, -0.248335249, -0.658459852, 0.188615233], 'A4GALT': [-0.630383282, 2.070597417, -0.406911403, 0.514365161, 0.1837369], 'A4GNT': [-0.349856602, 0.0, -1.429732702, 0.735813315, 1.527701557], 'AAAS': [0.012681166, -0.360123915, -0.122448127, -0.072317767, 0.014894341], 'AACS': [-0.248082823, -0.345460098, -0.145758741, -0.375537843, -0.254525913], 'AADAC': [-0.780512986, 0.641100995, 0.368972734, -0.075989113

Linked data shape after handling missing values: (31, 18381)
For the feature 'Rheumatoid_Arthritis', the least common label is '1.0' with 14 occurrences. This represents 45.16% of the dataset.
The distribution of the feature 'Rheumatoid_Arthritis' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 10 occurrences. This represents 32.26% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Usable linked data saved to ../../output/preprocess/Rheumatoid_Arthritis/GSE42842.csv
